# Data Cleaning

### 1. Read DataFrames

In [5]:
import csv
import pandas as pd

def read_activity_data():
    with open(r"data/activityDat.csv") as file:
        data = list(csv.reader(file))
        df = pd.DataFrame(data[1:], columns=data[0])
        df.columns = ['Year', 'Status', 'Provocation', 'Activity']
        print(df.head())
        return df


def read_injury_data():
    with open(r"data/injurydat.csv") as file:
        data = list(csv.reader(file))
        df = pd.DataFrame(data[1:], columns=data[0])
        df.columns = ['Day', 'Month', 'Year', 'Injury', 'State', 'Location', 'Latitude', 'Longitude', 'SharkName',
                      'SharkLength', 'Provocation', 'SharksCount', 'Activity', 'InjuryLocation', 'Severity', 'Gender',
                      'Age', 'IncidentTime']
        print(df.head())
        return df


def read_timedb2_data():
    with open(r"data/timedb2.csv") as file:
        data = list(csv.reader(file))
        df = pd.DataFrame(data[1:], columns=data[0])
        df.columns = ['Day', 'Month', 'Year', 'Latitude', 'Longitude', 'SharkName', 'SharkScientific',
                      'Provocation', 'Activity', 'InjuryLocation', 'InjuryDescription', 'Severity',
                      'Gender', 'Age', 'IncidentTime']
        print(df.head())
        return df


df_act = read_activity_data()
df_inj = read_injury_data()
df_time = read_timedb2_data()


   Year   Status Provocation  Activity
0  1791    fatal  unprovoked  swimming
1  1803  injured  unprovoked  swimming
2  1807  injured  unprovoked  swimming
3  1820    fatal    provoked  swimming
4  1825  injured  unprovoked  swimming
  Day Month  Year   Injury State                           Location  \
0   1     1  1791    fatal   NSW                        near sydney   
1  22     3  1803  injured    WA        "hamelin bay, faure island"   
2  18     1  1807  injured   NSW       "cockle bay, sydney harbour"   
3   1     1  1820    fatal   TAS     "sweetwater point, pitt water"   
4   1     1  1825  injured   NSW  "kirribili point, sydney harbour"   

       Latitude    Longitude    SharkName SharkLength Provocation SharksCount  \
0  -33.86666667        151.2  white shark              unprovoked               
1  -25.83333333  113.8833333  tiger shark              unprovoked           1   
2  -33.86666667        151.2   bull shark              unprovoked           1   
3         -42.8

### 2. Compare DataFrames

In [6]:
def compare_dataframes(df1, df2, df1_name='df1', df2_name='df2'):
    """
    Compare two dataframes and return detailed differences.
    
    Parameters:
    df1, df2: pandas DataFrames to compare
    df1_name, df2_name: names of the dataframes for clear reporting
    
    Returns:
    dict: Comparison results
    """
    comparison = {}
    
    # Compare basic properties
    comparison['shape'] = {
        'equal': df1.shape == df2.shape,
        df1_name: df1.shape,
        df2_name: df2.shape
    }
    
    # Compare column names
    comparison['columns'] = {
        'equal': set(df1.columns) == set(df2.columns),
        'only_in_' + df1_name: list(set(df1.columns) - set(df2.columns)),
        'only_in_' + df2_name: list(set(df2.columns) - set(df1.columns))
    }
    
    # Compare dtypes for common columns
    common_columns = list(set(df1.columns) & set(df2.columns))
    dtype_diff = {}
    for col in common_columns:
        if df1[col].dtype != df2[col].dtype:
            dtype_diff[col] = {
                df1_name: str(df1[col].dtype),
                df2_name: str(df2[col].dtype)
            }
    comparison['dtype_differences'] = dtype_diff
    
    # Compare data for common columns
    if comparison['shape']['equal'] and comparison['columns']['equal']:
        # Check if dataframes are exactly equal
        exact_match = df1.equals(df2)
        comparison['exact_match'] = exact_match
        
        if not exact_match:
            # Find rows with differences
            diff_mask = ~(df1 == df2).all(axis=1)
            diff_indices = diff_mask[diff_mask].index
            
            # Get sample of differences
            sample_diffs = {}
            for idx in diff_indices[:5]:  # Limit to first 5 differences
                row_diff = {}
                for col in df1.columns:
                    if df1.loc[idx, col] != df2.loc[idx, col]:
                        row_diff[col] = {
                            df1_name: df1.loc[idx, col],
                            df2_name: df2.loc[idx, col]
                        }
                sample_diffs[str(idx)] = row_diff
            
            comparison['differences'] = {
                'total_different_rows': len(diff_indices),
                'sample_differences': sample_diffs
            }
    
    return comparison

def compare_multiple_dataframes(dfs_dict):
    """
    Compare multiple dataframes against each other.
    
    Parameters:
    dfs_dict: Dictionary of dataframes with their names as keys
    
    Returns:
    dict: Pairwise comparison results
    """
    results = {}
    df_names = list(dfs_dict.keys())
    
    for i in range(len(df_names)):
        for j in range(i + 1, len(df_names)):
            name_1, name_2 = df_names[i], df_names[j]
            df1, df2 = dfs_dict[name_1], dfs_dict[name_2]
            
            comparison_key = f"{name_1}_vs_{name_2}"
            results[comparison_key] = compare_dataframes(df1, df2, name_1, name_2)
    
    return results

In [7]:
# Create a dictionary of your dataframes
dfs = {
    'activity': df_act,
    'injury': df_inj,
    'timedb2': df_time
}

# Compare all dataframes
results = compare_multiple_dataframes(dfs)

# Print results in a readable format
for comparison, result in results.items():
    print(f"\n=== {comparison} ===")
    
    # Print shape comparison
    print(f"\nShape comparison:")
    print(f"Equal shapes: {result['shape']['equal']}")
    print(f"Shapes: {result['shape']}")
    
    # Print column comparison
    print(f"\nColumn comparison:")
    print(f"Equal columns: {result['columns']['equal']}")
    if result['columns']['equal'] == False:
        print("Column differences:", result['columns'])
    
    # Print dtype differences if any
    if result['dtype_differences']:
        print("\nDatatype differences:", result['dtype_differences'])
    
    # Print data differences if applicable
    if 'exact_match' in result:
        print(f"\nExact match: {result['exact_match']}")
        if not result['exact_match']:
            print(f"Number of different rows: {result['differences']['total_different_rows']}")
            print("Sample of differences:", result['differences']['sample_differences'])


=== activity_vs_injury ===

Shape comparison:
Equal shapes: False
Shapes: {'equal': False, 'activity': (1196, 4), 'injury': (1196, 18)}

Column comparison:
Equal columns: False
Column differences: {'equal': False, 'only_in_activity': ['Status'], 'only_in_injury': ['Day', 'SharkName', 'SharkLength', 'IncidentTime', 'Injury', 'InjuryLocation', 'Location', 'Severity', 'State', 'Age', 'Latitude', 'Longitude', 'SharksCount', 'Month', 'Gender']}

=== activity_vs_timedb2 ===

Shape comparison:
Equal shapes: False
Shapes: {'equal': False, 'activity': (1196, 4), 'timedb2': (1196, 15)}

Column comparison:
Equal columns: False
Column differences: {'equal': False, 'only_in_activity': ['Status'], 'only_in_timedb2': ['Day', 'SharkName', 'IncidentTime', 'InjuryLocation', 'SharkScientific', 'Severity', 'Age', 'Latitude', 'Longitude', 'InjuryDescription', 'Month', 'Gender']}

=== injury_vs_timedb2 ===

Shape comparison:
Equal shapes: False
Shapes: {'equal': False, 'injury': (1196, 18), 'timedb2': (119

### 3. Merge DataFrames

In [10]:
def merge_shark_dataframes(df_activity, df_injury, df_timedb2):
    """
    Merge three shark incident dataframes while preserving all unique columns
    and validating the merge results.
    
    Parameters:
    df_activity: Activity dataframe
    df_injury: Injury dataframe
    df_timedb2: TimeDB2 dataframe
    
    Returns:
    pandas.DataFrame: Combined dataframe
    dict: Merge statistics
    """
    import pandas as pd
    
    # Create index if not exists (assuming rows correspond 1-to-1)
    for df, name in [(df_activity, 'activity'), (df_injury, 'injury'), (df_timedb2, 'timedb2')]:
        if not df.index.name:
            df.reset_index(drop=True, inplace=True)
    
    # Start with activity dataframe as base
    combined_df = df_activity.copy()
    
    # Dictionary to track source of each column
    column_sources = {col: 'activity' for col in df_activity.columns}
    
    # Merge with injury dataframe
    for col in df_injury.columns:
        if col not in combined_df.columns:
            combined_df[col] = df_injury[col]
            column_sources[col] = 'injury'
        else:
            # If column exists, check if values are the same
            if not df_injury[col].equals(combined_df[col]):
                combined_df[f'{col}_injury'] = df_injury[col]
                column_sources[f'{col}_injury'] = 'injury'
    
    # Merge with timedb2 dataframe
    for col in df_timedb2.columns:
        if col not in combined_df.columns:
            combined_df[col] = df_timedb2[col]
            column_sources[col] = 'timedb2'
        else:
            # If column exists, check if values are the same
            if not df_timedb2[col].equals(combined_df[col]):
                combined_df[f'{col}_timedb2'] = df_timedb2[col]
                column_sources[f'{col}_timedb2'] = 'timedb2'
    
    # Calculate merge statistics
    stats = {
        'total_columns': len(combined_df.columns),
        'columns_per_source': {
            'activity': sum(1 for source in column_sources.values() if source == 'activity'),
            'injury': sum(1 for source in column_sources.values() if source == 'injury'),
            'timedb2': sum(1 for source in column_sources.values() if source == 'timedb2')
        },
        'column_sources': column_sources,
        'duplicated_columns': [col for col in combined_df.columns if '_injury' in col or '_timedb2' in col]
    }
    
    return combined_df, stats

def print_merge_summary(stats):
    """Print a readable summary of the merge statistics."""
    print("\nMerge Summary:")
    print(f"Total columns in combined dataset: {stats['total_columns']}")
    print("\nColumns from each source:")
    for source, count in stats['columns_per_source'].items():
        print(f"- {source}: {count} columns")
    
    if stats['duplicated_columns']:
        print("\nDuplicated columns (may have different values):")
        for col in stats['duplicated_columns']:
            print(f"- {col}")
    
    print("\nColumn sources:")
    for col, source in stats['column_sources'].items():
        print(f"- {col}: from {source}")
        
# Merge the dataframes
combined_df, merge_stats = merge_shark_dataframes(df_act, df_inj, df_time)

# Print merge summary
print_merge_summary(merge_stats)

# Display basic info about the combined dataframe
print("\nCombined DataFrame Info:")
print(combined_df.info())


Merge Summary:
Total columns in combined dataset: 22

Columns from each source:
- activity: 4 columns
- injury: 15 columns
- timedb2: 3 columns

Duplicated columns (may have different values):
- Latitude_timedb2

Column sources:
- Year: from activity
- Status: from activity
- Provocation: from activity
- Activity: from activity
- Day: from injury
- Month: from injury
- Injury: from injury
- State: from injury
- Location: from injury
- Latitude: from injury
- Longitude: from injury
- SharkName: from injury
- SharkLength: from injury
- SharksCount: from injury
- InjuryLocation: from injury
- Severity: from injury
- Gender: from injury
- Age: from injury
- IncidentTime: from injury
- Latitude_timedb2: from timedb2
- SharkScientific: from timedb2
- InjuryDescription: from timedb2

Combined DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1196 entries, 0 to 1195
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------

In [11]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1196 entries, 0 to 1195
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Year               1196 non-null   object
 1   Status             1196 non-null   object
 2   Provocation        1196 non-null   object
 3   Activity           1196 non-null   object
 4   Day                1196 non-null   object
 5   Month              1196 non-null   object
 6   Injury             1196 non-null   object
 7   State              1196 non-null   object
 8   Location           1196 non-null   object
 9   Latitude           1196 non-null   object
 10  Longitude          1196 non-null   object
 11  SharkName          1196 non-null   object
 12  SharkLength        1196 non-null   object
 13  SharksCount        1196 non-null   object
 14  InjuryLocation     1196 non-null   object
 15  Severity           1196 non-null   object
 16  Gender             1196 non-null   object


### 4. Remove Duplicates

In [24]:
duplicate_count = combined_df.duplicated().sum()
print(f"Duplicate count before removing: {duplicate_count}")
new_df = combined_df.copy().drop_duplicates()
duplicate_count_new = new_df.duplicated().sum()
print(f"Duplicate count after removing: {duplicate_count_new}")

Duplicate count before removing: 2
Duplicate count after removing: 0


In [25]:
combined_df.drop_duplicates()

,Year,Status,Provocation,Activity,Day,Month,Injury,State,Location,Latitude,...,SharkLength,SharksCount,InjuryLocation,Severity,Gender,Age,IncidentTime,Latitude_timedb2,SharkScientific,InjuryDescription
0,1791,fatal,unprovoked,swimming,1,1,fatal,NSW,near sydney,-33.86666667,...,,,torso,major lacerations,female,,,-33.86666667,Carcharodon carcharias,
1,1803,injured,unprovoked,swimming,22,3,injured,WA,"""hamelin bay, faure island""",-25.83333333,...,,1,,,male,,,-25.83333333,Galeocerdo cuvier,
2,1807,injured,unprovoked,swimming,18,1,injured,NSW,"""cockle bay, sydney harbour""",-33.86666667,...,,1,"""arm, hand""",minor lacerations,male,,,-33.86666667,Carcharhinus leucas,
3,1820,fatal,provoked,swimming,1,1,fatal,TAS,"""sweetwater point, pitt water""",-42.8,...,,1,leg,major lacerations,male,,,-42.8,,
4,1825,injured,unprovoked,swimming,1,1,injured,NSW,"""kirribili point, sydney harbour""",-33.85,...,,1,leg,minor lacerations,male,15,,-33.85,Carcharhinus leucas,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,2022,injured,unprovoked,other:floating,17,2,injured,QLD,"""Redcliffe, Brisbane""",-27.2333,...,3.5,1,leg,major lacerations,,,,-27.2333,Galeocerdo cuvier,
1192,2022,injured,unprovoked,other:floating,6,2,injured,WA,"""wylie bay, esperance""",-33.84848�,...,3.3,1,torso,major lacerations,female,20,1200,-33.84848,Carcharodon carcharias,
1193,2022,injured,unprovoked,scuba diving,20,2,injured,NSW,"""shelley beach, manly""",-33.8006,...,,1,,,,,,-33.8006,Orectolobidae,
1194,2022,injured,unprovoked,wading,22,3,injured,NSW,crowdy head beach,-32,...,,1,minor lacerations,male,32,swimwear,,-32,Orectolobidae,3 lacerations on lower calf muscle


### 5. Handle missing values

In [26]:
def check_empty_values(df):
    results = pd.DataFrame({
        'null_count': df.isnull().sum(),
        'empty_string_count': (df.astype(str) == '').sum(),
        'none_string_count': (df.astype(str) == 'None').sum(),
        'nan_string_count': (df.astype(str) == 'nan').sum(),
        'unknown_count': df.apply(lambda x: (x.astype(str).str.lower() == 'unknown').sum())
    })
    
    # Add percentage columns
    total_rows = len(df)
    for col in results.columns:
        results[f'{col}_percent'] = (results[col] / total_rows * 100).round(2)
    
    # Filter to show only columns with any type of empty values
    has_empty = results.sum(axis=1) > 0
    results = results[has_empty].sort_values('null_count', ascending=False)
    
    return results

# Run the check
empty_analysis = check_empty_values(combined_df)
print("\nColumns with any type of empty values:")
print(empty_analysis)

# Show a sample of values for columns with potential empty values
for col in empty_analysis.index:
    print(f"\nValue counts for {col}:")
    print(combined_df[col].value_counts(dropna=False).head())


Columns with any type of empty values:
                   null_count  empty_string_count  none_string_count  \
Provocation                 0                   4                  0   
Activity                    0                  25                  0   
Day                         0                   1                  0   
Location                    0                   3                  0   
SharkName                   0                  47                  0   
SharkLength                 0                 617                  0   
SharksCount                 0                  89                  0   
InjuryLocation              0                 129                  0   
Severity                    0                 345                  0   
Gender                      0                  17                  0   
Age                         0                 497                  0   
IncidentTime                0                 683                  0   
SharkScientific         

In [27]:
def clean_shark_data(df):
    df_clean = df.copy()
    
    # 1. High missing values (>50%) - These need special consideration
    # InjuryDescription (99.67% empty) - Drop this column as it's almost entirely empty
    df_clean = df_clean.drop('InjuryDescription', axis=1)
    
    # IncidentTime (57.11% empty) - Create a category for unknown time
    df_clean['IncidentTime'] = df_clean['IncidentTime'].replace('', 'Unknown')
    
    # SharkLength (51.59% empty) - Convert to float and fill with median
    df_clean['SharkLength'] = pd.to_numeric(df_clean['SharkLength'].replace('', np.nan), errors='coerce')
    df_clean['SharkLength'] = df_clean['SharkLength'].fillna(df_clean['SharkLength'].median())
    
    # 2. Moderate missing values (20-50%)
    # Age (41.56% empty) - Convert to numeric and fill with median
    df_clean['Age'] = pd.to_numeric(df_clean['Age'].replace('', np.nan), errors='coerce')
    df_clean['Age'] = df_clean['Age'].fillna(df_clean['Age'].median())
    
    # Severity (28.85% empty) - Create 'Unknown' category
    df_clean['Severity'] = df_clean['Severity'].replace('', 'Unknown')
    
    # 3. Low missing values (<20%)
    # InjuryLocation (10.79% empty) - Create 'Unknown' category
    df_clean['InjuryLocation'] = df_clean['InjuryLocation'].replace('', 'Unknown')
    
    # SharksCount (7.44% empty) - Fill with 1 (most common value)
    df_clean['SharksCount'] = df_clean['SharksCount'].replace('', 1)
    
    # SharkScientific (4.26% empty) and SharkName (3.93% empty)
    # Fill with 'Unknown species'
    df_clean['SharkScientific'] = df_clean['SharkScientific'].replace('', 'Unknown species')
    df_clean['SharkName'] = df_clean['SharkName'].replace('', 'Unknown species')
    
    # 4. Very low missing values (<3%)
    # Activity (2.09% empty)
    df_clean['Activity'] = df_clean['Activity'].replace('', 'Unknown')
    
    # Gender (1.42% empty) - Fill with 'Unknown'
    df_clean['Gender'] = df_clean['Gender'].replace(['', '32', '47'], 'Unknown')
    
    # Provocation (0.33% empty) - Fill with most common category
    mode_provocation = df_clean['Provocation'].mode()[0]
    df_clean['Provocation'] = df_clean['Provocation'].replace('', mode_provocation)
    
    # Location (0.25% empty) - Fill with 'Unknown location'
    df_clean['Location'] = df_clean['Location'].replace('', 'Unknown location')
    
    # Day (0.08% empty) - Fill with median day
    df_clean['Day'] = pd.to_numeric(df_clean['Day'].replace('', np.nan), errors='coerce')
    df_clean['Day'] = df_clean['Day'].fillna(df_clean['Day'].median())
    
    return df_clean

# Apply the cleaning
import numpy as np
cleaned_df = clean_shark_data(combined_df)

# Verify the cleaning
print("\nMissing values after cleaning:")
print(cleaned_df.isnull().sum())

# Show basic statistics of numeric columns
print("\nNumeric columns statistics:")
print(cleaned_df.describe())


Missing values after cleaning:
Year                0
Status              0
Provocation         0
Activity            0
Day                 0
Month               0
Injury              0
State               0
Location            0
Latitude            0
Longitude           0
SharkName           0
SharkLength         0
SharksCount         0
InjuryLocation      0
Severity            0
Gender              0
Age                 0
IncidentTime        0
Latitude_timedb2    0
SharkScientific     0
dtype: int64

Numeric columns statistics:
               Day  SharkLength          Age
count  1196.000000  1196.000000  1196.000000
mean     14.466555     2.647659    26.825251
std       9.280008     0.842532    10.640983
min       1.000000     0.300000     0.000000
25%       6.000000     2.600000    22.000000
50%      14.000000     2.600000    25.000000
75%      23.000000     2.600000    28.000000
max      31.000000     6.000000    84.000000


### 6. Handle outliers and datatypes

In [28]:
def clean_data_further(df):
    df_clean = df.copy()
    
    # 1. Fix Data Types
    # Convert numeric columns
    numeric_cols = ['Age', 'SharkLength', 'SharksCount', 'Day']
    for col in numeric_cols:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    # Convert IncidentTime to datetime format if possible
    df_clean['IncidentTime'] = pd.to_datetime(df_clean['IncidentTime'], format='%H%M', errors='coerce')
    
    # 2. Standardize Text Data
    text_cols = ['Activity', 'Location', 'SharkName', 'Provocation', 'Severity', 'Gender']
    for col in text_cols:
        df_clean[col] = df_clean[col].str.strip().str.lower()
    
    # 3. Remove Special Characters and Fix Formatting
    df_clean['Location'] = df_clean['Location'].str.replace('"', '')
    
    # 4. Check and Fix Inconsistent Categories
    # Example for Activity column
    df_clean['Activity'] = df_clean['Activity'].replace({
        'swimming ': 'swimming',
        'swim': 'swimming',
        'surfing': 'boarding',
        'surf': 'boarding',
        # Add more replacements as needed
    })
    
    # 5. Create Derived Features
    # Extract time of day category
    def get_time_category(time):
        if pd.isna(time):
            return 'unknown'
        hour = time.hour
        if 5 <= hour < 12:
            return 'morning'
        elif 12 <= hour < 17:
            return 'afternoon'
        elif 17 <= hour < 21:
            return 'evening'
        else:
            return 'night'
    
    df_clean['TimeOfDay'] = df_clean['IncidentTime'].apply(get_time_category)
    
    # 6. Handle Outliers
    # Example for Age
    Q1 = df_clean['Age'].quantile(0.25)
    Q3 = df_clean['Age'].quantile(0.75)
    IQR = Q3 - Q1
    df_clean['Age'] = df_clean['Age'].clip(lower=Q1 - 1.5*IQR, upper=Q3 + 1.5*IQR)
    
    return df_clean

# Apply the cleaning
cleaned_df = clean_data_further(cleaned_df)

# Check results
print("\nData Types:")
print(cleaned_df.dtypes)

print("\nUnique values in categorical columns:")
categorical_cols = ['Activity', 'Provocation', 'Severity', 'Gender', 'TimeOfDay']
for col in categorical_cols:
    print(f"\n{col}:", cleaned_df[col].unique())

print("\nSummary statistics for numeric columns:")
print(cleaned_df.describe())


Data Types:
Year                        object
Status                      object
Provocation                 object
Activity                    object
Day                        float64
Month                       object
Injury                      object
State                       object
Location                    object
Latitude                    object
Longitude                   object
SharkName                   object
SharkLength                float64
SharksCount                float64
InjuryLocation              object
Severity                    object
Gender                      object
Age                        float64
IncidentTime        datetime64[ns]
Latitude_timedb2            object
SharkScientific             object
TimeOfDay                   object
dtype: object

Unique values in categorical columns:

Activity: ['swimming' 'fishing' 'unknown' 'spearfishing' 'unmotorised boating'
 'snorkelling' 'diving' 'motorised boating' 'other: standing in water'
 'boarding' '

In [33]:
# Add data validation checks
def validate_data(df):
    validations = {
        'Age within range': (df['Age'] >= 0) & (df['Age'] <= 150),
        'SharkLength is positive': df['SharkLength'] > 0,
        'SharksCount is positive': df['SharksCount'] > 0,
    }
    
    for check, condition in validations.items():
        invalid_count = (~condition).sum()
        if invalid_count > 0:
            print(f"Failed {check}: {invalid_count} records")
            
validate_data(cleaned_df)

### 7. Export to CSV file

In [35]:
cleaned_df.to_csv("cleaned_data.csv", index=False)